In [2]:
# Base packages
import numpy as np
import pandas as pd
import itertools
import warnings
from tqdm import tqdm

# Data Visualization
from matplotlib import pyplot as plt
import seaborn as sns

# Modelling
import pmdarima as pm
from statsmodels.tsa.api import ExponentialSmoothing
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Evaluation
from sklearn.metrics import mean_absolute_error

In [3]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.width", 500)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.float_format", lambda x: '%.1f' % x)
warnings.filterwarnings('ignore')

## Data Manipulation

In [4]:
data = pd.read_csv("C:/Users/MONSTER/Desktop/train.csv")

def get_season(month):
    if month in [1, 2, 3]:
        return 'Q1'
    elif month in [4, 5, 6]:
        return 'Q2'
    elif month in [7, 8, 9]:
        return 'Q3'
    else: # 10, 11, 12
        return 'Q4'

data['month_id'] = data['month_id'].astype(str)
data['year'] = data['month_id'].str[:4].astype(int)
data['month'] = data['month_id'].str[4:].astype(int)
data['season'] = data['month'].apply(get_season)
data['month_id'] = pd.to_datetime(data['month_id'], format='%Y%m')


column_order = ['month_id', 'merchant_id', 'year', 'month', 'season', 'merchant_source_name', 'settlement_period', 'working_type', 'mcc_id', 'merchant_segment', 'net_payment_count']
data = data[column_order]
data = data.sort_values(by='month_id').reset_index(drop=True)

# Segment column
data.loc[data['merchant_segment'].isin(["Segment - 1", "Segment - 2", "Segment - 3"]), 'merchant_segment'] = "otherSegments123"
data.loc[data['merchant_segment'] == "Segment - 4", 'merchant_segment'] = "StandartSegment4"
# negative values in target 
data.loc[data['net_payment_count'] < 0, 'net_payment_count'] = 0
# mcc_id to integer
data['mcc_id'] = data['mcc_id'].str.replace('mcc_', '').astype(int)
# Working type
data.loc[data['working_type'].isin(["Working Type - 1", "Working Type - 3", "Working Type - 4"]), 'working_type'] = "otherWorkingTypes134"
# settlement_period
data.loc[data['settlement_period'].isin(["Settlement Period - 2", "Settlement Period - 3"]), 'settlement_period'] = "otherSettlementPeriods23"

In [5]:
data.head(10)

,month_id,merchant_id,year,month,season,merchant_source_name,settlement_period,working_type,mcc_id,merchant_segment,net_payment_count
0,2020-01-01,merchant_66740,2020,1,Q1,Merchant Source - 1,Settlement Period - 1,Working Type - 6,130,StandartSegment4,6
1,2020-01-01,merchant_12444,2020,1,Q1,Merchant Source - 1,Settlement Period - 1,Working Type - 2,153,StandartSegment4,3
2,2020-01-01,merchant_40154,2020,1,Q1,Merchant Source - 2,Settlement Period - 1,Working Type - 5,168,StandartSegment4,3
3,2020-01-01,merchant_33179,2020,1,Q1,Merchant Source - 3,otherSettlementPeriods23,Working Type - 2,25,otherSegments123,787
4,2020-01-01,merchant_16977,2020,1,Q1,Merchant Source - 2,Settlement Period - 1,Working Type - 6,31,StandartSegment4,5
5,2020-01-01,merchant_25991,2020,1,Q1,Merchant Source - 2,Settlement Period - 1,Working Type - 6,110,StandartSegment4,8
6,2020-01-01,merchant_92,2020,1,Q1,Merchant Source - 1,Settlement Period - 1,Working Type - 6,73,StandartSegment4,3
7,2020-01-01,merchant_13117,2020,1,Q1,Merchant Source - 1,Settlement Period - 1,Working Type - 6,126,StandartSegment4,9
8,2020-01-01,merchant_43413,2020,1,Q1,Merchant Source - 1,Settlement Period - 1,Working Type - 6,42,StandartSegment4,14
9,2020-01-01,merchant_26670,2020,1,Q1,Merchant Source - 1,Settlement Period - 1,Working Type - 2,166,StandartSegment4,3


In [6]:
data["merchant_id"].nunique()

26060

In [7]:
time_series = data.groupby(["month_id", "merchant_id"]).agg({"net_payment_count": "sum"}).reset_index()

In [8]:
time_series.tail()

,month_id,merchant_id,net_payment_count
291137,2023-09-01,merchant_9946,3
291138,2023-09-01,merchant_9965,5
291139,2023-09-01,merchant_9982,5
291140,2023-09-01,merchant_9984,5
291141,2023-09-01,merchant_9991,4


In [9]:
time_series.shape

(291142, 3)

In [10]:
time_series.groupby(["merchant_id", "month_id"]).agg({"net_payment_count": "sum"}).reset_index().head(10)

,merchant_id,month_id,net_payment_count
0,merchant_1,2021-08-01,3
1,merchant_1,2022-10-01,3
2,merchant_10,2021-11-01,6
3,merchant_10,2021-12-01,24
4,merchant_10,2022-01-01,15
5,merchant_10,2022-02-01,7
6,merchant_10,2022-03-01,9
7,merchant_10,2022-04-01,4
8,merchant_10000,2022-05-01,3
9,merchant_10000,2022-06-01,4


In [11]:
time_series.loc[time_series["merchant_id"] == "merchant_2419"].head(100).sort_values(by="month_id")

,month_id,merchant_id,net_payment_count
849,2020-01-01,merchant_2419,10
4608,2020-02-01,merchant_2419,32
8499,2020-03-01,merchant_2419,11
12524,2020-04-01,merchant_2419,511
16752,2020-05-01,merchant_2419,2274
21497,2020-06-01,merchant_2419,1751
26438,2020-07-01,merchant_2419,1975
31492,2020-08-01,merchant_2419,4231
36795,2020-09-01,merchant_2419,5184
42317,2020-10-01,merchant_2419,7808


In [12]:
# Create a date range for all months
all_months = pd.date_range(start='2020-01-01', end='2023-09-01', freq='MS')

# Create a DataFrame with all combinations of customer_id and all_months
all_customers = time_series['merchant_id'].unique()
all_combinations = pd.MultiIndex.from_product([all_customers, all_months], names=['merchant_id', 'month_id'])
complete_df = pd.DataFrame(index=all_combinations).reset_index()

# Merge the complete_df with the original df
df_merged = complete_df.merge(time_series, on=['merchant_id', 'month_id'], how='left')

# Fill missing sales with zero
df_merged['net_payment_count'] = df_merged['net_payment_count'].fillna(0)


In [41]:
df_merged.head(10)

,merchant_id,month_id,net_payment_count
0,merchant_10005,2020-01-01,3.0
1,merchant_10005,2020-02-01,4.0
2,merchant_10005,2020-03-01,3.0
3,merchant_10005,2020-04-01,0.0
4,merchant_10005,2020-05-01,0.0
5,merchant_10005,2020-06-01,4.0
6,merchant_10005,2020-07-01,3.0
7,merchant_10005,2020-08-01,0.0
8,merchant_10005,2020-09-01,3.0
9,merchant_10005,2020-10-01,4.0


## Modelling

In [13]:
def tes_optimizer(train, test, abg, step, logging=False):
    best_alpha, best_beta, best_gamma, best_mae = None, None, None, float("inf")

    for comb in abg:
        tes_model = ExponentialSmoothing(train, trend="add", seasonal="add", seasonal_periods=12).\
            fit(smoothing_level=comb[0], smoothing_slope=comb[1], smoothing_seasonal=comb[2])
        
        y_pred = tes_model.forecast(step)
        mae = mean_absolute_error(test, y_pred)

        if mae < best_mae:
            best_alpha, best_beta, best_gamma, best_mae = comb[0], comb[1], comb[2], mae
        if logging:
            print([round(comb[0], 2), round(comb[1], 2), round(comb[2], 2), round(mae, 2)])

    if logging:
        print("best_alpha:", round(best_alpha, 2), "best_beta:", round(best_beta, 2), "best_gamma:", round(best_gamma, 2),
              "best_mae:", round(best_mae, 4))

    return best_alpha, best_beta, best_gamma, best_mae


In [30]:
forecast_df = pd.DataFrame(columns=['month_id', 'merchant_id', 'forecast'])

alphas = betas = gammas = np.arange(0.4, 1, 0.4)

abg = list(itertools.product(alphas, betas, gammas))

iteration_control = 0
total_mae = 0

for customer_id in tqdm(df_merged['merchant_id'].unique(), desc="Processing Customers"):
    try:
        customer_data = df_merged[df_merged['merchant_id'] == customer_id]
        customer_data = customer_data.set_index('month_id')
        customer_data = customer_data.drop('merchant_id', axis=1)

        # Ensure the sales column is numeric and the index is datetime
        customer_data['net_payment_count'] = pd.to_numeric(customer_data['net_payment_count'], errors='coerce')
        customer_data.index = pd.to_datetime(customer_data.index)

        # Split the data into training and testing
        train = customer_data[:"2023-05-01"]
        test = customer_data["2023-06-01":]

        # Optimize the model
        y_pred_len = len(test)
        best_alpha, best_beta, best_gamma, best_mae = tes_optimizer(train, test, abg, y_pred_len, logging=False)

        # Train the model
        tes_model = ExponentialSmoothing(customer_data, trend="add", seasonal="add", seasonal_periods=12).\
            fit(smoothing_level=best_alpha, smoothing_slope=best_beta, smoothing_seasonal=best_gamma)
        
        # Forecast
        y_pred = tes_model.forecast(3)
        y_pred = pd.DataFrame(y_pred, columns=['forecast'])

        # print(f"Customer {customer_id} - MAE: {best_mae}")

        # Ensuring the forecast is 1-dimensional
        y_pred = np.array(y_pred).flatten()

        # Creating a new DataFrame for the forecasted values
        forecast_dates = pd.date_range(start=customer_data.index[-1], periods=len(y_pred) + 1, freq='MS')[1:]
        forecast_df_temp = pd.DataFrame({
            'month_id': forecast_dates,
            'merchant_id': [customer_id] * len(y_pred),  # Repeating the customer_id for each prediction
            'forecast': y_pred
        })

        # Append the forecasts for this customer to the forecast_df
        forecast_df = forecast_df.append(forecast_df_temp, ignore_index=True)

        iteration_control += 1
        total_mae += best_mae

    except Exception as e:
        print(f"Error processing in customer: {customer_id}: {e}")
        break

# After all forecasts have been appended to forecast_df
avg_mae = total_mae / iteration_control
forecast_df['forecast'] = forecast_df['forecast'].apply(lambda x: 0 if x < 1 else x)
print(f"Average MAE: {avg_mae}")

Processing Customers: 100%|██████████| 26060/26060 [2:26:17<00:00,  2.97it/s]  

Average MAE: 30.76022206280839


In [31]:
forecast_df_new = forecast_df.copy()

In [32]:
forecast_df_new.head()

,month_id,merchant_id,forecast
0,2023-10-01,merchant_10005,1.1
1,2023-11-01,merchant_10005,0.0
2,2023-12-01,merchant_10005,1.4
3,2023-10-01,merchant_10034,0.0
4,2023-11-01,merchant_10034,0.0


In [33]:
forecast_df_new['date_id'] = forecast_df_new['month_id'].apply(lambda x: x.strftime('%Y%m') if x.month in [10, 11, 12] else '')


In [34]:
forecast_df_new['general_merchant_id'] = forecast_df_new['date_id'] + forecast_df_new['merchant_id']

In [35]:
forecast_df_new.head()

,month_id,merchant_id,forecast,date_id,general_merchant_id
0,2023-10-01,merchant_10005,1.1,202310,202310merchant_10005
1,2023-11-01,merchant_10005,0.0,202311,202311merchant_10005
2,2023-12-01,merchant_10005,1.4,202312,202312merchant_10005
3,2023-10-01,merchant_10034,0.0,202310,202310merchant_10034
4,2023-11-01,merchant_10034,0.0,202311,202311merchant_10034


In [36]:
forecast_df_new.to_csv("C:/Users/MONSTER/Desktop/forecast.csv", index=False)

In [74]:
last_three_months = df_merged['month_id'].max() - pd.DateOffset(months=3)
new_customer_df = df_merged[df_merged['month_id'] < last_three_months].groupby('merchant_id').filter(lambda x: x['net_payment_count'].sum() == 0)
new_customer_ids = new_customer_df['merchant_id'].unique()

In [78]:
new_customer_ids

array(['merchant_57693', 'merchant_1010', 'merchant_10211', ...,
       'merchant_9373', 'merchant_9383', 'merchant_9545'], dtype=object)

In [81]:
forecast_df = pd.DataFrame(columns=['month_id', 'merchant_id', 'forecast'])

alphas = betas = gammas = np.arange(0.4, 1, 0.4)

abg = list(itertools.product(alphas, betas, gammas))

iteration_control = 0
total_mae = 0

for customer_id in tqdm(df_merged['merchant_id'].unique(), desc="Processing Customers"):
    try:
        customer_data = df_merged[df_merged['merchant_id'] == customer_id]
        customer_data = customer_data.set_index('month_id')
        customer_data = customer_data.drop('merchant_id', axis=1)

        # Ensure the sales column is numeric and the index is datetime
        customer_data['net_payment_count'] = pd.to_numeric(customer_data['net_payment_count'], errors='coerce')
        customer_data.index = pd.to_datetime(customer_data.index)

        if customer_id in new_customer_ids:
            moving_average_october = (customer_data['net_payment_count']["2023-09-01"] + customer_data['net_payment_count']["2023-08-01"] + customer_data['net_payment_count']["2023-07-01"] + customer_data['net_payment_count']["2023-06-01"]) / 4
            moving_average_november = (customer_data['net_payment_count']["2023-09-01"] + customer_data['net_payment_count']["2023-08-01"] + customer_data['net_payment_count']["2023-07-01"]) / 3 
            moving_average_december = (customer_data['net_payment_count']["2023-09-01"] + customer_data['net_payment_count']["2023-08-01"]) / 2
            
            moving_averages_df = pd.DataFrame({
                'month_id': ["2023-10-01", "2023-11-01", "2023-12-01"],
                'merchant_id': [customer_id] * 3,
                'forecast': [moving_average_october, moving_average_november, moving_average_december]
            })
            
            forecast_df = forecast_df.append(moving_averages_df, ignore_index=True)
            
        else:
            # Split the data into training and testing
            train = customer_data[:"2023-05-01"]
            test = customer_data["2023-06-01":]

            # Optimize the model
            y_pred_len = len(test)
            best_alpha, best_beta, best_gamma, best_mae = tes_optimizer(train, test, abg, y_pred_len, logging=False)

            # Train the model
            tes_model = ExponentialSmoothing(customer_data, trend="add", seasonal="add", seasonal_periods=12).\
                fit(smoothing_level=best_alpha, smoothing_slope=best_beta, smoothing_seasonal=best_gamma)
            
            # Forecast
            y_pred = tes_model.forecast(3)
            y_pred = pd.DataFrame(y_pred, columns=['forecast'])

            # print(f"Customer {customer_id} - MAE: {best_mae}")

            # Ensuring the forecast is 1-dimensional
            y_pred = np.array(y_pred).flatten()

            # Creating a new DataFrame for the forecasted values
            forecast_dates = pd.date_range(start=customer_data.index[-1], periods=len(y_pred) + 1, freq='MS')[1:]
            forecast_df_temp = pd.DataFrame({
                'month_id': forecast_dates,
                'merchant_id': [customer_id] * len(y_pred),  # Repeating the customer_id for each prediction
                'forecast': y_pred
            })

            # Append the forecasts for this customer to the forecast_df
            forecast_df = forecast_df.append(forecast_df_temp, ignore_index=True)

            iteration_control += 1
            total_mae += best_mae

    except Exception as e:
        print(f"Error processing in customer: {customer_id}: {e}")
        continue

# After all forecasts have been appended to forecast_df
avg_mae = total_mae / iteration_control
forecast_df['forecast'] = forecast_df['forecast'].apply(lambda x: 0 if x < 1 else x)
print(f"Average MAE: {avg_mae}")

Processing Customers:  17%|█▋        | 4473/26060 [10:58<45:46,  7.86it/s]  

     month_id     merchant_id  forecast
0  2023-10-01  merchant_57693       0.0
1  2023-11-01  merchant_57693       0.0
2  2023-12-01  merchant_57693       0.0


Processing Customers:  94%|█████████▍| 24598/26060 [59:07<01:55, 12.64it/s] 

     month_id    merchant_id  forecast
0  2023-10-01  merchant_1010       1.5
1  2023-11-01  merchant_1010       1.0
2  2023-12-01  merchant_1010       1.5
     month_id     merchant_id  forecast
0  2023-10-01  merchant_10211       3.5
1  2023-11-01  merchant_10211       3.7
2  2023-12-01  merchant_10211       3.5
     month_id     merchant_id  forecast
0  2023-10-01  merchant_10264       0.8
1  2023-11-01  merchant_10264       0.0
2  2023-12-01  merchant_10264       0.0
     month_id     merchant_id  forecast
0  2023-10-01  merchant_10455       8.8
1  2023-11-01  merchant_10455      10.0
2  2023-12-01  merchant_10455      11.0
     month_id    merchant_id  forecast
0  2023-10-01  merchant_1054       3.0
1  2023-11-01  merchant_1054       3.0
2  2023-12-01  merchant_1054       2.0


Processing Customers:  94%|█████████▍| 24602/26060 [59:07<01:33, 15.61it/s]

...

     month_id    merchant_id  forecast
0  2023-10-01  merchant_7872       0.8
1  2023-11-01  merchant_7872       1.0
2  2023-12-01  merchant_7872       1.5
     month_id    merchant_id  forecast
0  2023-10-01  merchant_8117       0.8
1  2023-11-01  merchant_8117       1.0
2  2023-12-01  merchant_8117       1.5
     month_id    merchant_id  forecast
0  2023-10-01  merchant_8224       4.0
1  2023-11-01  merchant_8224       5.3
2  2023-12-01  merchant_8224       8.0
     month_id    merchant_id  forecast
0  2023-10-01  merchant_8295       0.8
1  2023-11-01  merchant_8295       1.0
2  2023-12-01  merchant_8295       1.5


Processing Customers: 100%|█████████▉| 26054/26060 [1:00:30<00:00, 20.22it/s]

     month_id    merchant_id  forecast
0  2023-10-01  merchant_8635       8.5
1  2023-11-01  merchant_8635      11.3
2  2023-12-01  merchant_8635      17.0
     month_id  merchant_id  forecast
0  2023-10-01  merchant_88       0.8
1  2023-11-01  merchant_88       1.0
2  2023-12-01  merchant_88       1.5
     month_id   merchant_id  forecast
0  2023-10-01  merchant_902       0.8
1  2023-11-01  merchant_902       1.0
2  2023-12-01  merchant_902       1.5
     month_id    merchant_id  forecast
0  2023-10-01  merchant_9100       1.0
1  2023-11-01  merchant_9100       1.3
2  2023-12-01  merchant_9100       2.0
     month_id    merchant_id  forecast
0  2023-10-01  merchant_9177       1.0
1  2023-11-01  merchant_9177       1.3
2  2023-12-01  merchant_9177       2.0


Processing Customers: 100%|██████████| 26060/26060 [1:00:30<00:00,  7.18it/s]

     month_id    merchant_id  forecast
0  2023-10-01  merchant_9338       1.0
1  2023-11-01  merchant_9338       1.3
2  2023-12-01  merchant_9338       2.0
     month_id    merchant_id  forecast
0  2023-10-01  merchant_9373       0.8
1  2023-11-01  merchant_9373       1.0
2  2023-12-01  merchant_9373       1.5
     month_id    merchant_id  forecast
0  2023-10-01  merchant_9383       0.8
1  2023-11-01  merchant_9383       1.0
2  2023-12-01  merchant_9383       1.5
     month_id    merchant_id  forecast
0  2023-10-01  merchant_9545       4.0
1  2023-11-01  merchant_9545       5.3
2  2023-12-01  merchant_9545       8.0
Average MAE: 39.69889154904894


In [82]:
avg_mae

39.69889154904894

In [83]:
forecast_df_new = forecast_df.copy()

In [84]:
forecast_df_new.head()

,month_id,merchant_id,forecast
0,2023-10-01 00:00:00,merchant_10005,0.0
1,2023-11-01 00:00:00,merchant_10005,0.0
2,2023-12-01 00:00:00,merchant_10005,1.0
3,2023-10-01 00:00:00,merchant_10034,0.0
4,2023-11-01 00:00:00,merchant_10034,1.6


In [85]:
forecast_df_new["month_id"] = pd.to_datetime(forecast_df_new["month_id"], format='%Y-%m-%d')

In [86]:
forecast_df_new.head()

,month_id,merchant_id,forecast
0,2023-10-01,merchant_10005,0.0
1,2023-11-01,merchant_10005,0.0
2,2023-12-01,merchant_10005,1.0
3,2023-10-01,merchant_10034,0.0
4,2023-11-01,merchant_10034,1.6


In [87]:
forecast_df_new['date_id'] = forecast_df_new['month_id'].apply(lambda x: x.strftime('%Y%m') if x.month in [10, 11, 12] else '')
forecast_df_new['general_merchant_id'] = forecast_df_new['date_id'] + forecast_df_new['merchant_id']

In [88]:
forecast_df_new.head()

,month_id,merchant_id,forecast,date_id,general_merchant_id
0,2023-10-01,merchant_10005,0.0,202310,202310merchant_10005
1,2023-11-01,merchant_10005,0.0,202311,202311merchant_10005
2,2023-12-01,merchant_10005,1.0,202312,202312merchant_10005
3,2023-10-01,merchant_10034,0.0,202310,202310merchant_10034
4,2023-11-01,merchant_10034,1.6,202311,202311merchant_10034


In [90]:
forecast_df_new.to_csv("C:/Users/MONSTER/Desktop/forecast.csv", index=False)